# Stable diffusion with Diffusers

This is a notebook that demonstrates how to use the [Diffusers package](https://huggingface.co/docs/diffusers/index) from Huggingface to run stable diffusion. [Huggingface](https://huggingface.co) is a community-driven  platform that provides a comprehensive suite of open-source libraries and tools for machine learning. Think of it as a kind of "github for machine learning". The diffusers package provides a simple API and a number of pretrained models that allow to easily run and experiment with diffusion models. 

**NOTE** this notebook should work on other platform, but it has been only tested on Mac (M1). 

## Installation
To use this notebook you will need to install diffusers with
```
pip install --upgrade diffusers\[torch\]
```


## Running stable diffusion
To run stable diffusion you will need to distinguish wether you are running this on a Mac M1/M2 or on Linux/Windows with a Nvidia GPU. 
On a M1/M2 Mac, diffusion will need a "warmup" phase to work properly (see [this link](https://huggingface.co/docs/diffusers/optimization/mps))

In [ ]:
import platform
if platform.processor() == 'arm':
    print("Detected Mac M1/M2")
    device = 'mps'
else:
    device = 'cuda'
print(device)

Now this will load the pretrained model and download it the first time the cell is run (which might take a while). You can set the model version by modifying the `sd_version` variable. 

In [ ]:
from diffusers import StableDiffusionPipeline

sd_version = '2.1'

if sd_version == '2.1':
    model_key = "stabilityai/stable-diffusion-2-1-base"
elif sd_version == '2.0':
    model_key = "stabilityai/stable-diffusion-2-base"
elif sd_version == '1.5':
    model_key = "runwayml/stable-diffusion-v1-5"

generator = StableDiffusionPipeline.from_pretrained(model_key)
generator = generator.to(device)

In [ ]:
if device=='mps':
    # Recommended if your computer has < 64 GB of RAM
    generator.enable_attention_slicing()


Generate the image (note increasing `num_inference_steps` will improve quality but be slower)

In [ ]:
prompt = "A cubist painting of the Star Treck character Spock, high quality, trending on artstation"
image = generator(prompt, guidance_scale=7.5, num_inference_steps=20).images[0]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.imshow(np.array(image))
plt.title(prompt)
plt.axis('off')
plt.show()

## Conditioning stable diffusion with ControlNet

[ControlNet](https://stablediffusionweb.com/ControlNet) is a very recent and quite amazing advancement in image generation using stable diffusion. It allows conditioning the stable diffusion generation pipeline on an image input (similarly to pix2pix).

If you are running a Mac, you will need to update torch to the "nightly" version with
```
conda update pytorch torchvision -c pytorch-nightly
```
(see here: https://github.com/huggingface/diffusers/issues/2521)

Let's use skimage to create edges from an input imgage.

In [ ]:
import numpy as np
from skimage import io, feature, transform
import PIL.Image as Image
import cv2

def apply_canny_skimage(img, sigma=1.5, size=512):
    import cv2
    from skimage import feature
    invert = False
    grayimg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = (feature.canny(grayimg, sigma=sigma)*255).astype(np.uint8)
    if invert:
        edges = cv2.bitwise_not(edges)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

img = io.imread("images/spock.jpg")
edges =  apply_canny_skimage(img)

# ControlNet expects a PIL Image as input
edges_image = Image.fromarray(edges)

plt.figure(figsize=(8,4))
plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(edges)
plt.show()

Setup the ControlNet model. Somehow converting to device takes ages on Mac. Here we use the canny-to-image model, there are others available: 
[https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/controlnet](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/controlnet)

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch
import PIL.Image as Image

controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", 
    torch_dtype=torch.float16)
    
generator = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", # Controlnet is currently working only with SD 1.5
    torch_dtype=torch.float16,
    controlnet=controlnet, 
    safety_checker=None)
print("converting to device")
generator = generator.to(device)


In [ ]:
prompt = "A sculpture made of clay on a white background"

if device=='mps':
    generator.enable_attention_slicing()
    #_ = generator(prompt, image=edges_image, num_inference_steps=1)
elif device != 'cuda':
    # this command loads the individual model components on GPU on-demand.
    generator.enable_model_cpu_offload()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

image = generator(
    prompt,
    image=edges_image,
    num_inference_steps=30,
).images[0]

plt.imshow(np.array(image))
plt.title(prompt)
plt.axis('off')
plt.show()